In [5]:
! python --version
# Pour éviter le code d'installation ci-dessous, la version de Python doit être 3.8

Python 3.11.11


In [7]:
# --------------------------------------------
# 1) Installation d'Experta dans le notebook
# --------------------------------------------
! pip uninstall frozendict experta --yes
! pip install frozendict>=2.3.4   # on installe manuellement la version la plus récente
! pip install experta --no-deps   # on installe Experta sans vérifier ses deps (qui pin frozendict==1.2)

Found existing installation: frozendict 1.2
Uninstalling frozendict-1.2:
  Successfully uninstalled frozendict-1.2
Found existing installation: experta 1.9.4
Uninstalling experta-1.9.4:
  Successfully uninstalled experta-1.9.4
  Using cached experta-1.9.4-py3-none-any.whl.metadata (5.0 kB)
Using cached experta-1.9.4-py3-none-any.whl (35 kB)


In [8]:
from experta import *

# ---------------------------------------------------------------------
# 1) Définition des classes de faits
# ---------------------------------------------------------------------
class Truck(Fact):
    """
    Représente un camion :
      - wheels: nombre de roues (0, 1 ou 2)
      - driver: nom du chauffeur (ou None si pas de chauffeur)
    """
    pass

class StockWheel(Fact):
    """
    Représente le stock de roues disponibles :
      - count: nombre de roues encore libres en réserve
    """
    pass

class Driver(Fact):
    """
    Représente un chauffeur :
      - name: nom du chauffeur
      - is_free: booléen indiquant s'il est libre ou non
    """
    pass

# ---------------------------------------------------------------------
# 2) Moteur de règles
# ---------------------------------------------------------------------
class TruckEngine(KnowledgeEngine):
    """
    Applique 2 règles principales :
      - R1: si un camion a <2 roues ET qu'il existe un stockWheel.count>0
            => on ajoute 1 roue au camion, on décrémente le stock.
      - R2: si un camion a 2 roues, pas de chauffeur,
            ET qu'il existe un chauffeur libre => on assigne ce chauffeur au camion.
    """

    # R1: Ajouter une roue manquante
    @Rule(
        AS.t_fact << Truck(wheels=MATCH.w, driver=MATCH.d),
        AS.s_fact << StockWheel(count=MATCH.c),
        TEST(lambda w, c: w < 2 and c > 0)
    )
    def add_wheel(self, t_fact, s_fact, w, d, c):
        """
        - Retire l'ancien camion "t_fact" (ex: wheels=1) et le stock "s_fact" (ex: 2 roues disponibles).
        - Reconstruit un nouveau camion avec 1 roue de plus (w+1).
        - Reconstruit un stockWheel avec count-1 roue.
        """
        self.retract(t_fact)
        self.retract(s_fact)
        self.declare(Truck(wheels=w + 1, driver=d))
        self.declare(StockWheel(count=c - 1))
        print(f"[R1] Ajout d'une roue => Le camion passe à {w+1} roue(s), stock restant: {c-1}")

    # R2: Assigner un chauffeur si 2 roues et pas de driver
    @Rule(
        AS.t_fact << Truck(wheels=2, driver=L(None)),
        AS.dr_fact << Driver(name=MATCH.nom, is_free=True)
    )
    def assign_driver(self, t_fact, dr_fact, nom):
        """
        - Retire l'ancien Truck sans driver
        - Retire le chauffeur libre
        - Crée un nouveau Truck avec driver=nom
        - Déclare le chauffeur non libre
        """
        self.retract(t_fact)
        self.retract(dr_fact)
        self.declare(Truck(wheels=2, driver=nom))
        self.declare(Driver(name=nom, is_free=False))
        print(f"[R2] Affectation du chauffeur '{nom}' au camion => Il peut rouler !")

# ---------------------------------------------------------------------
# 3) Utilisation / Démo
# ---------------------------------------------------------------------
# a) On crée l'engine et on le réinitialise
engine = TruckEngine()
engine.reset()

# b) On déclare un stock initial de roues, des camions et des chauffeurs
#    (un exemple où le camion1 a 1 roue, le camion2 a 0 roue, etc.)
engine.declare(StockWheel(count=2))            # 2 roues en réserve
engine.declare(Truck(wheels=1, driver=None))   # camion incomplet
engine.declare(Truck(wheels=2, driver=None))   # camion complet mais pas de chauffeur
engine.declare(Truck(wheels=2, driver="Bob"))  # camion déjà complet et avec chauffeur
engine.declare(Driver(name="Alice", is_free=True))
engine.declare(Driver(name="Charlie", is_free=True))

# c) On exécute le chaînage avant (les règles R1 et R2 s'appliqueront si possible)
engine.run()

# d) On inspecte les faits finaux
print("\n=== Faits finaux dans la base ===")
for fact_id, fact_val in engine.facts.items():
    print(f"{fact_id:2} : {fact_val}")

[R2] Affectation du chauffeur 'Charlie' au camion => Il peut rouler !
[R1] Ajout d'une roue => Le camion passe à 2 roue(s), stock restant: 1
[R2] Affectation du chauffeur 'Alice' au camion => Il peut rouler !

=== Faits finaux dans la base ===
 0 : <f-0>
 4 : <f-4>
 7 : <f-7>
 8 : <f-8>
10 : <f-10>
11 : <f-11>
12 : <f-12>
